# Análisis Acceso a la Salud Bucal en Barrios Populares de la Ciudad de Corrientes, Argentina
---
### Resumen:
El presente representa un análisis exploratorio de las consultas odontológicas realizadas en los distintos saps de la localidad de Corrientes desde el año 2020.
Los datos utilizados fueron proporcionados por el Portal de Datos Abiertos.

In [1]:
# Importamos las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import folium
from shapely.geometry import shape
import plotly.express as px

In [452]:
# guardamos los datos en una variable
df = pd.read_csv('consultas_odontologicas - Odontologia.csv')

# Ubicación de los saps
saps = pd.read_csv('Odata.xlsm - SAPS.csv')


# Ubicación de los barrios
geo_barrios = gpd.read_file('Odata.xlsm - Barrios.csv')
df_barrios = pd.read_csv('Odata.xlsm - Barrios.csv')

# Ubicación de los barrios populares
barrios_populares = pd.read_csv('bdgis_RENABAP - bdgis_RENABAP.csv')

In [319]:
# Mapeo para unir cada saps con su ubicación geográfica

mapeo_saps = {
    'DR. BENJAMIN SERRANO': 'SAP XVII - Bº Parque Cadenas Norte',
    'DR. ANIBAL MALVIDO': 'SAP XX - Dr. A. Malvido',
    'DR. SUSSINI': 'SAP XXXIII - Dr. Sussini',
    'DR MANUEL A. GONZALEZ': 'SAP XIV - Dr. M. Gonzales',
    'DR. JOSE DE LOS REYES VIDAL': 'SAP XXVIII - Dr. De Los Reyes Vidal',
    'ITATI': 'SAP IV - Bº Itatí',
    'PRIMERA JUNTA': 'SAP XIX - Bº Primera Junta', 
    'DR. PEDRO BLUGERMAN': 'SAP XXXIII - Dr. Blugerman',
    'MARCELINO VERA': 'SAP XXI - Cañada Quiroz',
    'DR. SEMPER': 'SAP XV - Dr. Semper', 
    'DR. ALBERTO LIFSCHITZ': 'SAP XXV - Dr. Lifchitz',
    'DR. GUILLERMO RAWSON': 'SAP I - Dr. G. Rawson',
    'QUINTA FERRE': 'SAP V - Bº Quinta Ferre',
    'VILLA GARCIA': 'SAP II - Villa García',
    'RIO PARANA': 'SAP XXXI - Bº Río Parana',
    'SANTA MARTA': 'SAPS XXIX - Bº Santa Marta',
    'DR. PIRCHI': 'SAP III - Bº Anahí',
    'GÜEMES': 'SAP XXXVII - Bº Guemes',
    'ESPERANZA': 'SAP XII - Bº Esperanza',
    'Dr. CALMANASH': 'SAP XXX - Dr. Calmanash',
    'SANTA MARGARITA': 'SAP XIII - Bº Santa Margarita',
    'SAN MARCOS': 'SAP VII - Bº San Marcos',
    'DR. MANUEL CASSUSO': 'SAP IX - Dr. M. Casusso',
    'DR. KORIMBLUM': 'SAP X - Dr. Korimblum',
    'DR. ROSSI CANDIA': 'SAP VIII - Dr. Rossi Candia',
    'Dr. SEMPER': 'SAP XV - Dr. Semper',
    'DR. FLIER': 'SAPS XXVII - Dr. Juan Carlos Benitez Suarez',
    'Dr. SANTIAGO LORENZO': 'SAP XXVI - Dr. Santiago Lorenzo',
    'PASO MARTINEZ': 'SAPS XVI - Paso Martinez',
    'CICHERO': 'SAP XXXV - Bº Cichero' 
}

# Aplicar el mapeo al dataset df para obtener los nombres correctos de los SAPS
df['nombre_correcto'] = df['saps'].map(mapeo_saps)

In [320]:
# unir los dos dataframes de consultas y saps usando la columna 'nombre_correcto' en df y 'nombre' en saps
# para cruzar los datos

# Selecciona solo las columnas que necesitas del dataframe saps
saps_reducido = saps[['nombre', 'lng', 'lat', 'barrio']]

# Ahora realiza el merge con df
df = pd.merge(df, saps_reducido, left_on='nombre_correcto', right_on='nombre', how='left')

# Elimina la columna 'nombre' si no la necesitas
df = df.drop(columns=['nombre'])

In [321]:
# Cruzamos con el dataset barrios de esa manera obtenemos los polígonos, primero mapeamos los barrios
# para que no haya problemas
# mapeo al dataset df para obtener los nombres correctos de los Barrios de los SAPS

mapa_barrios = {
    'CHICHERO': 'CICHERO',
    'LOMAS DEL MIRADOR': 'LOMAS DEL MIRADOR',
    'SAN ANTONIO ESTE': 'SAN ANTONIO',
    'PIRAYUI': 'PIRAYUI',
    'PARQUE CADENAS': 'PARQUE CADENA NORTE',
    'LOMAS':'LOMAS',
    'SAN GERONIMO': 'SAN GERONIMO',
    'SANTA RITA': 'SANTA RITA NORTE',
    'ONGAY': 'VILLA ONGAY',
    'SAN ANTONIO': 'SAN ANTONIO',
    'VIRGEN DE LOS DOLORES': 'VIRGEN DE LOS DOLORES',
    'ANAHI': 'ANAHI',
    'INDEPENDENCIA': 'INDEPENDENCIA',
    'VICTOR COLAS': 'VICTOR COLAS',
    'PEDRO FERRÉ': 'PEDRO FERRE',
    'QUILMES':'VILLA PATONO',
    'NUEVO':'NTRA SRA DE LA ASUNCION',
    'VICTOR COLAS':'VICTOR COLAS',
    'ESPERANZA': 'ESPERANZA',
    'GUEMES': 'GUEMES TRADICIONAL',
    'ITATÍ':'PLACIDO MARTINEZ',
    'CAÑADA QUIROZ':'',
    'PASO MARTINEZ': 'PASO MARTINEZ',
    'PRIMERA JUNTA': 'PRIMERA JUNTA',
    'QUINTA FERRÉ': 'QUINTA FERRE',
    'RIO PARANÁ': 'RIO PARANA',
    'SAN MARCOS': 'SAN MARCOS',
    'SANTA MARGARITA': 'SANTA MARGARITA',
    '3 DE ABRIL': '3 DE ABRIL',
    'VILLA GARCÍA': 'NIÑO JESUS',
}

df['barrio'] = df['barrio'].str.upper()

df['nombre_barrio_correcto'] = df['barrio'].map(mapa_barrios)

In [322]:
# Mergeamos con el dataset barrios que tiene la información de los polígonos de los barrios
# Seleccionar solo las columnas necesarias del DataFrame barrios
barrios_reducido = barrios[['nombre_barrio', 'sup_ha', 'st_asgeojson']]

# Convertir 'nombre_barrio' a mayúsculas para asegurar la coincidencia al hacer el merge
barrios_reducido = barrios[['nombre_barrio', 'sup_ha', 'st_asgeojson']].copy()
barrios_reducido['nombre_barrio'] = barrios_reducido['nombre_barrio'].str.upper()

# Realizar el merge
df = pd.merge(df, barrios_reducido, left_on='nombre_barrio_correcto', right_on='nombre_barrio', how='left')

In [323]:
# Por último antes de transformar nuestro dataset, vamos a determinar si el sap corresponde a un barrio popular

dic_sap_populares = {
    'SAP XXI - Cañada Quiroz':'no', 
    'SAP XII - Bº Esperanza': 'si',
    'SAPS XXVII - Dr. Juan Carlos Benitez Suarez': 'no',
    'SAP I - Dr. G. Rawson': 'no', 
    'SAP XV - Dr. Semper': 'no',
    'SAP XIV - Dr. M. Gonzales' : 'si', 
    'SAP VI - Bº Pujol' : 'si',
    'SAP IV - Bº Itatí': 'si', 
    'SAP II - Villa García': 'no',
    'SAP XI - Bº Villa Chiquita': 'no', 
    'SAP XXXVII - Bº Guemes': 'no',
    'SAP VII - Bº San Marcos': 'no', 
    'SAP XIX - Bº Primera Junta': 'no',
    'SAP X - Dr. Korimblum': 'si', 
    'SAP XX - Dr. A. Malvido': 'no',
    'SAP XXVI - Dr. Santiago Lorenzo' : 'si', 
    'SAP XXV - Dr. Lifchitz': 'si',
    'SAP IX - Dr. M. Casusso': 'no', 
    'SAP VIII - Dr. Rossi Candia': 'si',
    'SAPS XXIX - Bº Santa Marta' : 'si', 
    'SAP XXXIII - Dr. Blugerman': 'si',
    'SAPS XXXIV - Galván': 'si',
    'SAP XXXIII - Dr. Sussini': 'no',
    'SAP XIII - Bº Santa Margarita': 'si', 
    'SAPS XVIII - Bº Sapucay' : 'si',
    'SAPS XVI - Paso Martinez': 'no', 
    'SAP XXXVI- Centro de Vacunación': 'no',
    'SAP V - Bº Quinta Ferre':'si',
    'SAP XXXI - Bº Río Parana': 'si',
    'SAP XXX - Dr. Calmanash': 'si', 
    'SAP III - Bº Anahí': 'no',
    'SAP XXVIII - Dr. De Los Reyes Vidal':'si',
    'SAP XVII - Bº Parque Cadenas Norte': 'no', 
    'SAPS SANTA CATALINA': 'no',
    'SAPS XXIV- Mauricio Open': 'no', 
    'SAP XXIII - Dr. Chercoff': 'no',
    'SAP XXXV - Bº Cichero': 'si'   
}

# Mapea los valores de la columna 'nombre_correcto' con el diccionario 'dic_sap_populares' para obtener si
# el saps está en un barrio popular o no
df['barrio_popular'] = df_final['nombre_correcto'].map(dic_sap_populares)

# Si hay valores NaN (es decir, nombres que no están en el diccionario), puedes reemplazarlos con 'no'
df['barrio_popular'].fillna('no', inplace=True)


In [324]:
# Vamos a proceder a limpiar un poco los datos, y eliminar columnas que no nos sirvan

# Primero la columna rango_etario

# Definir una función para normalizar los valores de 'rango_etario'
def normalizar_rango_etario(rango):
    # Verificar si 'rango' es una cadena
    if isinstance(rango, str):
        # Reemplazar 'ANIOS' por 'ANOS'
        rango = rango.replace('ANIOS', 'ANOS')
        # Eliminar espacios adicionales
        rango = rango.replace(' - ', '-')
        # Otros reemplazos específicos
        rango = rango.replace('>=70 ANOS', '>=70 ANOS')
        rango = rango.replace('< 1 ANO', '<1 ANO')
        rango = rango.replace('< 1 ANIO', '<1 ANO')
        rango = rango.replace('#¡REF!', 'Desconocido')
    else:
        # Si 'rango' no es una cadena, simplemente devolver 'Desconocido' o alguna otra etiqueta
        rango = 'Desconocido'
    # Devolver el rango normalizado
    return rango

# Aplicar la función a la columna 'rango_etario'
df['rango_etario_normalizado'] = df['rango_etario'].apply(normalizar_rango_etario)


In [325]:
eliminar_columnas = ['saps', 'rango_etario', 'barrio', 'nombre_barrio']

# Eliminar las columnas especificadas
df = df.drop(columns=eliminar_columnas)

In [327]:
df = df[['fecha', 'nombre_correcto', 'servicio', 'turno', 'rango_etario_normalizado',
         'consulta_cantidad', 'sexo', 'detalle', 'nombre_barrio_correcto',
         'barrio_popular', 'sup_ha', 'lng', 'lat', 'st_asgeojson']]

In [367]:
fecha_dic = {'20221-11-03': '2022-11-03',
             '20201-11-30': '2020-11-30',
             '20212-11-01': '2021-11-01',
             '25021-07-01': '2021-07-01',
             '20321-12-01': '2021-12-01',
             '20212-12-01': '2021-12-01',
             '20231-10-01': '2023-10-01',
             '20231-10-01': '2023-10-01',
             '20231-02-11': '2023-02-11',
             '20225-01-01': '2022-01-01',
             '2023-10-01': '2023-01-01',
             
            }
# Reemplazar las fechas incorrectas en el DataFrame
df['fecha'] = df['fecha'].replace(fecha_dic)

# Convertir 'fecha' en datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Filtra el DataFrame para mantener solo las filas donde el año no es 2011
df = df[df['fecha'].dt.year != 2011]

/tmp/ipykernel_114274/3176402158.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_114274/3176402158.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [368]:
df.to_csv('dataframe_saps_limpio.csv')

In [330]:
df_sin_geo = df[['fecha', 'nombre_correcto', 'servicio', 'turno', 'rango_etario_normalizado',
         'consulta_cantidad', 'sexo', 'detalle', 'nombre_barrio_correcto',
         'barrio_popular', 'sup_ha', 'lng', 'lat']]

df_sin_geo.to_csv('df_saps_sin_geo.csv')

In [369]:
df.fecha.unique()

array(['2020-02-01T00:00:00.000000000', '2020-02-02T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-05-01T00:00:00.000000000', '2020-06-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2020-08-01T00:00:00.000000000',
       '2020-09-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2020-10-10T00:00:00.000000000', '2020-11-01T00:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000',
       '2020-01-01T00:00:00.000000000', '2021-02-01T00:00:00.000000000',
       '2021-02-10T00:00:00.000000000', '2021-03-01T00:00:00.000000000',
       '2021-03-02T00:00:00.000000000', '2021-03-31T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-04-20T00:00:00.000000000',
       '2021-04-30T00:00:00.000000000', '2021-05-01T00:00:00.000000000',
       '2021-05-31T00:00:00.000000000', '2021-06-01T00:00:00.000000000',
       '2021-06-02T00:00:00.000000000', '2021-06-30

In [316]:


df['fecha'] = df['fecha'].map(fecha_dic)




,fecha,nombre_correcto,servicio,turno,rango_etario_normalizado,consulta_cantidad,sexo,detalle,nombre_barrio_correcto,barrio_popular,sup_ha,lng,lat,st_asgeojson
0,NaN,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
1,NaN,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,8.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
2,NaN,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,15.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
3,NaN,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,14.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
4,NaN,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,15-19 ANOS,1.0,F,CONSULTA-ADULTO (Desde 15 anios),PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22974,NaN,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,<1 ANO,1.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22975,NaN,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22976,NaN,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,3.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22977,NaN,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,5-14 ANOS,2.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."


### Total de consultas. Comparación entre SAPS
---
Cantidad de consultas por saps desde el año 2020.

In [345]:
departamentos = gpd.read_file('departamentos.geojson')

departamentos

,nombre_completo,fuente,nombre,id,provincia,categoria,geometry
0,Partido de José C. Paz,ARBA - Gerencia de Servicios Catastrales,José C. Paz,06412,"{'nombre': 'Buenos Aires', 'id': '06', 'inters...",Partido,POINT (-58.77767 -34.51188)
1,Departamento O'Higgins,Direc. Pcial. de Catastro y Cartografía,O'Higgins,22112,"{'nombre': 'Chaco', 'id': '22', 'interseccion'...",Departamento,POINT (-60.68068 -27.25755)
2,Departamento Trenel,Direc. Grl. de Catastro,Trenel,42147,"{'nombre': 'La Pampa', 'id': '42', 'intersecci...",Departamento,POINT (-64.21438 -35.63188)
3,Partido de Adolfo Gonzales Chaves,ARBA - Gerencia de Servicios Catastrales,Adolfo Gonzales Chaves,06014,"{'nombre': 'Buenos Aires', 'id': '06', 'inters...",Partido,POINT (-60.24908 -37.96423)
4,Partido de General Juan Madariaga,ARBA - Gerencia de Servicios Catastrales,General Juan Madariaga,06315,"{'nombre': 'Buenos Aires', 'id': '06', 'inters...",Partido,POINT (-57.23079 -37.15321)
...,...,...,...,...,...,...,...
524,Departamento Capital,Direc. de Geodesia y Catastro,Capital,70028,"{'nombre': 'San Juan', 'id': '70', 'intersecci...",Departamento,POINT (-68.53429 -31.53307)
525,Departamento Chapaleufú,Direc. Grl. de Catastro,Chapaleufú,42056,"{'nombre': 'La Pampa', 'id': '42', 'intersecci...",Departamento,POINT (-63.66091 -35.22724)
526,Departamento Maracó,Direc. Grl. de Catastro,Maracó,42105,"{'nombre': 'La Pampa', 'id': '42', 'intersecci...",Departamento,POINT (-63.66249 -35.67962)
527,Departamento Concepción,IGN,Concepción,18028,"{'nombre': 'Corrientes', 'id': '18', 'intersec...",Departamento,POINT (-58.03138 -28.40840)


In [335]:
dpto_ctes = departamentos[departamentos['id'] == '18021']

In [338]:
mun_ctes = gpd.read_file('municipios(1).geojson')

In [346]:
mun_ctes = mun_ctes[mun_ctes['nombre_completo'] == 'Municipio Corrientes']

In [344]:
mun_ctes.to_file('mun_ctes.geojson', driver='GeoJSON')
dpto_ctes.to_file('dpto_ctes.geojson', driver='GeoJSON')

In [352]:
df

,fecha,nombre_correcto,servicio,turno,rango_etario_normalizado,consulta_cantidad,sexo,detalle,nombre_barrio_correcto,barrio_popular,sup_ha,lng,lat,st_asgeojson
0,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
1,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,8.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
2,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,15.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
3,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,14.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
4,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,15-19 ANOS,1.0,F,CONSULTA-ADULTO (Desde 15 anios),PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22974,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,<1 ANO,1.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22975,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22976,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,3.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22977,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,5-14 ANOS,2.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."


In [358]:
# Convertir 'fecha' en datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Filtra el DataFrame para mantener solo las filas donde el año no es 2011
df = df[df['fecha'].dt.year != 2011]

# Agrupa por 'barrio_popular' y mes y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

In [361]:
resultado=

array(['2020-01-01T00:00:00.000000000', '2020-02-01T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-05-01T00:00:00.000000000', '2020-06-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2020-08-01T00:00:00.000000000',
       '2020-09-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2020-11-01T00:00:00.000000000', '2020-12-01T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000', '2021-02-01T00:00:00.000000000',
       '2021-03-01T00:00:00.000000000', '2021-04-01T00:00:00.000000000',
       '2021-05-01T00:00:00.000000000', '2021-06-01T00:00:00.000000000',
       '2021-07-01T00:00:00.000000000', '2021-08-01T00:00:00.000000000',
       '2021-09-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2021-11-01T00:00:00.000000000', '2021-12-01T00:00:00.000000000',
       '2022-01-01T00:00:00.000000000', '2022-02-01T00:00:00.000000000',
       '2022-03-01T00:00:00.000000000', '2022-04-01

In [370]:
import pandas as pd
import plotly.express as px

# Cargar el DataFrame 'df'

# Convertir 'fecha' en datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Agrupa por 'barrio_popular' y mes y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de líneas
fig = px.line(
    resultado, 
    x='fecha', 
    y='consulta_cantidad', 
    color='barrio_popular', 
    title='Atenciones en Salas de Atención Primaria por Barrio Popular', 
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    template='plotly_dark'
)

# Mostrar el gráfico
fig.show()

In [432]:
# Agrupa por 'barrio_popular' y mes y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de líneas
fig = px.line(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='barrio_popular',
    title='Atenciones en Salas de Atención Primaria por Barrio Popular',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    # Comentar o eliminar la siguiente línea para tener fondo blanco
    # template='plotly_dark'
)

# Configurando el eje X para que muestre un tick para cada mes
fig.update_xaxes(
    dtick="M1",  # Cada mes
    tickformat="%b\n%Y"  # Formato "Mes\nAño"
)

# Actualizar las líneas para que sean suaves
fig.update_traces(
    line=dict(shape='spline')  # spline para suavizar la línea
)

# Guardar el gráfico como un archivo HTML
fig.write_html("atenciones_por_barrio_popular.html")

fig

In [393]:
import pandas as pd
import plotly.express as px


# Agrupa por 'rango_etario_normalizado', 'fecha' y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'rango_etario_normalizado'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de líneas
fig = px.line(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',  # Cambiar aquí para usar 'rango_etario_normalizado' como color
    title='Atenciones en Salas de Atención Primaria por Rango Etario',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    # Comentar o eliminar la siguiente línea para tener fondo blanco
    # template='plotly_dark'
)

# Configurando el eje X para que muestre un tick para cada mes
fig.update_xaxes(
    dtick="M1",  # Cada mes
    tickformat="%b\n%Y"  # Formato "Mes\nAño"
)

# Actualizar las líneas para que sean suaves
fig.update_traces(
    line=dict(shape='spline')  # spline para suavizar la línea
)

# Mostrar el gráfico
fig.show()


In [433]:
import pandas as pd
import plotly.express as px

# Agrupa por 'rango_etario_normalizado', 'fecha' y 'barrio_popular', y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'rango_etario_normalizado', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de líneas
fig = px.line(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',  # Cambiar aquí para usar 'rango_etario_normalizado' como color
    facet_row='barrio_popular',  # Separar los gráficos por filas según 'barrio_popular'
    title='Atenciones en Salas de Atención Primaria por Rango Etario',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
)

# Configurando el eje X para que muestre un tick para cada mes
fig.update_xaxes(
    dtick="M1",  # Cada mes
    tickformat="%b\n%Y"  # Formato "Mes\nAño"
)

# Actualizar las líneas para que sean suaves
fig.update_traces(
    line=dict(shape='spline')  # spline para suavizar la línea
)

# Mostrar el gráfico
fig.show()


In [434]:
# Agrupa por 'rango_etario_normalizado', 'fecha' y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'rango_etario_normalizado'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()


# Crear un gráfico de áreas
fig = px.area(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',
    title='Atenciones en Salas de Atención Primaria por Rango Etario',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'}
)

# Configuración adicional aquí...

# Mostrar el gráfico
fig.show()

In [392]:
# Crear un gráfico de barras apiladas
fig = px.bar(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',
    title='Atenciones en Salas de Atención Primaria por Rango Etario',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'}
)

# Configuración adicional aquí...

# Mostrar el gráfico
fig.show()


In [435]:
import pandas as pd
import plotly.express as px
# Agrupación
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'rango_etario_normalizado', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de área
fig = px.area(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',  # Usar la nueva columna para el color
    facet_row='barrio_popular',  # Usar facet_row para colocar un gráfico encima del otro
    title='Atenciones en Salas de Atención Primaria por Rango Etario y Barrio Popular',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    height=600,  # Ajustar la altura
    width=800,  # Ajustar el ancho
)

# Configuraciones del eje X
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y"
)

# Mostrar el gráfico
fig.show()

In [436]:
import pandas as pd
import plotly.express as px

# Agrupación
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'rango_etario_normalizado', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de barras apiladas
fig = px.bar(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='rango_etario_normalizado',  # Usar la nueva columna para el color
    facet_row='barrio_popular',  # Usar facet_row para apilar las barras por 'barrio_popular'
    title='Atenciones en Salas de Atención Primaria por Rango Etario y Barrio Popular',
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    height=600,  # Ajustar la altura
    width=800,  # Ajustar el ancho
)

# Configuraciones del eje X
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y"
)

# Mostrar el gráfico
fig.show()

In [406]:
df

,fecha,nombre_correcto,servicio,turno,rango_etario_normalizado,consulta_cantidad,sexo,detalle,nombre_barrio_correcto,barrio_popular,sup_ha,lng,lat,st_asgeojson
0,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
1,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,8.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
2,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,15.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
3,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,14.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
4,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,15-19 ANOS,1.0,F,CONSULTA-ADULTO (Desde 15 anios),PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22974,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,<1 ANO,1.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22975,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22976,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,3.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22977,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,5-14 ANOS,2.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."


In [438]:
import pandas as pd
import plotly.express as px

# Agrupa por 'fecha', 'genero' y 'barrio_popular' y suma las 'consulta_cantidad'
resultado = df.groupby([df['fecha'].dt.to_period('M'), 'sexo', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Convertir 'fecha' de nuevo a datetime
resultado['fecha'] = resultado['fecha'].dt.to_timestamp()

# Crear un gráfico de líneas
fig = px.line(
    resultado,
    x='fecha',
    y='consulta_cantidad',
    color='sexo',  # Usar 'genero' como color
    facet_col='barrio_popular',  # Dividir el gráfico por 'barrio_popular'
    labels={'consulta_cantidad': 'Cantidad de Consultas', 'fecha': 'Fecha'},
    # Comentar o eliminar la siguiente línea para tener fondo blanco
    # template='plotly_dark'
)

# Configurar el diseño del gráfico
fig.update_xaxes(
    dtick="M1",  # Cada mes
    tickformat="%b\n%Y"  # Formato "Mes\nAño" para el eje X
)

# Actualizar las líneas para que sean suaves
fig.update_traces(
    line=dict(shape='spline')  # spline para suavizar la línea
)

# Mostrar el gráfico
fig.show()

In [440]:
import plotly.express as px

# Agrupar los datos por 'barrio_popular' y sumar la 'consulta_cantidad'
atenciones_barrios_populares = df.groupby('barrio_popular')['consulta_cantidad'].sum().reset_index()

# Crear un gráfico de pie
fig = px.pie(
    atenciones_barrios_populares,
    names='barrio_popular',
    values='consulta_cantidad',
    title='Cantidad Total de Atenciones en Barrios Populares vs. No Populares',
    color='barrio_popular',  # Usar colores predeterminados para 'si' y 'no',
)

# Mostrar el gráfico
fig.show()


In [412]:
df

,fecha,nombre_correcto,servicio,turno,rango_etario_normalizado,consulta_cantidad,sexo,detalle,nombre_barrio_correcto,barrio_popular,sup_ha,lng,lat,st_asgeojson
0,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
1,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,8.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
2,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,15.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
3,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,14.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
4,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,15-19 ANOS,1.0,F,CONSULTA-ADULTO (Desde 15 anios),PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22974,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,<1 ANO,1.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22975,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22976,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,3.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22977,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,5-14 ANOS,2.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."


In [414]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# Procesar los datos para obtener la cantidad de consultas por barrio popular
barrios_populares = df[df['barrio_popular'] == 'si']
barrios_no_populares = df[df['barrio_popular'] == 'no']

# Calcular la cantidad de consultas por barrio
consultas_por_barrio_popular = barrios_populares.groupby('nombre_barrio_correcto')['consulta_cantidad'].sum().reset_index()
consultas_por_barrio_no_popular = barrios_no_populares.groupby('nombre_barrio_correcto')['consulta_cantidad'].sum().reset_index()

# Calcular el SAP con más consultas
sap_con_mas_consultas = df.groupby('nombre_correcto')['consulta_cantidad'].sum().idxmax()

# Calcular el SAP de barrios populares con más consultas
sap_barrios_populares_con_mas_consultas = barrios_populares.groupby('nombre_correcto')['consulta_cantidad'].sum().idxmax()

# Crear el gráfico
fig = px.bar(consultas_por_barrio_popular, x='nombre_barrio_correcto', y='consulta_cantidad', title='Consultas por Barrio Popular', labels={'nombre_barrio_correcto': 'Barrio', 'consulta_cantidad': 'Cantidad de Consultas'})
fig.update_xaxes(title_text='Barrio')
fig.update_yaxes(title_text='Cantidad de Consultas')
fig.show()

In [448]:
import plotly.express as px

# Agrupar los datos por 'barrio_popular' y sumar la 'consulta_cantidad'
atenciones_barrios_populares = df.groupby('barrio_popular')['consulta_cantidad'].sum().reset_index()

# Crear un gráfico de pastel
fig = px.pie(
    atenciones_barrios_populares,
    names='barrio_popular',
    values='consulta_cantidad',
    title='Cantidad Total de Atenciones en Barrios Populares vs. No Populares',
    color='barrio_popular',  # Usar colores predeterminados para 'si' y 'no'
)

# Personalizar las etiquetas de texto para mostrar el porcentaje y la cantidad con un solo decimal
fig.update_traces(
    textinfo='percent+label+value',  # Mostrar porcentaje, etiqueta y valor
    textposition='inside',  # Colocar el texto dentro del sector del pastel
    texttemplate='%{percent:.1%} (%{value})',  # Formato personalizado para el texto
)

# Mostrar el gráfico
fig.show()



In [416]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# ... (procesa los datos como se mostró anteriormente)

# Crear el gráfico Plotly con leyendas y colores
fig = px.bar(consultas_por_barrio_popular, x='nombre_barrio_correcto', y='consulta_cantidad', 
             title='Consultas por Barrio Popular',
             labels={'nombre_barrio_correcto': 'Barrio', 'consulta_cantidad': 'Cantidad de Consultas'},
             color='nombre_barrio_correcto')  # Colorear por barrio para una mejor lectura
fig.update_xaxes(title_text='Barrio')
fig.update_yaxes(title_text='Cantidad de Consultas')

# Añadir leyendas y título
fig.update_layout(legend_title_text='Barrio')
fig.update_layout(legend=dict(title=dict(text='Barrios')))
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Guardar la imagen en formato PNG (puedes cambiar el formato si lo deseas)
fig.write_image("grafico_consultas_mejorado.png")

# Crear un gráfico de torta para la cantidad total de consultas en barrios populares y no populares
consultas_totales = df.groupby('barrio_popular')['consulta_cantidad'].sum().reset_index()
fig_pie = px.pie(consultas_totales, names='barrio_popular', values='consulta_cantidad',
                 title='Cantidad Total de Consultas en Barrios Populares y No Populares')
fig_pie.update_traces(textinfo='percent+label')
fig_pie.write_image("grafico_torta_consultas.png")

print(f'SAP con más consultas: {sap_con_mas_consultas}')
print(f'SAP de barrios populares con más consultas: {sap_barrios_populares_con_mas_consultas}')


SAP con más consultas: SAP XV - Dr. Semper
SAP de barrios populares con más consultas: SAP XXV - Dr. Lifchitz


In [419]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# ... (procesa los datos como se mostró anteriormente)

# Crear el gráfico Plotly con leyendas y colores
fig = px.bar(consultas_por_barrio_popular, x='nombre_barrio_correcto', y='consulta_cantidad', 
             title='Consultas por Barrio Popular',
             labels={'nombre_barrio_correcto': 'Barrio', 'consulta_cantidad': 'Cantidad de Consultas'},
             color='nombre_barrio_correcto')  # Colorear por barrio para una mejor lectura
fig.update_xaxes(title_text='Barrio')
fig.update_yaxes(title_text='Cantidad de Consultas')

# Añadir leyendas y título
fig.update_layout(legend_title_text='Barrio')
fig.update_layout(legend=dict(title=dict(text='Barrios')))
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Mostrar el gráfico interactivo
fig.show()

# Guardar la imagen en formato PNG (puedes cambiar el formato si lo deseas)
fig.write_image("grafico_consultas_mejorado.png")

# Crear un gráfico de torta para la cantidad total de consultas en barrios populares y no populares
consultas_totales = df.groupby('barrio_popular')['consulta_cantidad'].sum().reset_index()
fig_pie = px.pie(consultas_totales, names='barrio_popular', values='consulta_cantidad',
                 title='Cantidad Total de Consultas en Barrios Populares y No Populares')
fig_pie.update_traces(textinfo='percent+label')

# Mostrar el gráfico de torta interactivo
fig_pie.show()

# Guardar el gráfico de torta como imagen PNG
fig_pie.write_image("grafico_torta_consultas.png")

print(f'SAP con más consultas: {sap_con_mas_consultas}')
print(f'SAP de barrios populares con más consultas: {sap_barrios_populares_con_mas_consultas}')


SAP con más consultas: SAP XV - Dr. Semper
SAP de barrios populares con más consultas: SAP XXV - Dr. Lifchitz


In [422]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# ... (procesa los datos como se mostró anteriormente)

# Calcular la cantidad de consultas por SAP
consultas_por_sap = df.groupby('nombre_correcto')['consulta_cantidad'].sum().reset_index()

# Ordenar el DataFrame de mayor a menor cantidad de consultas
consultas_por_sap = consultas_por_sap.sort_values(by='consulta_cantidad', ascending=False)

# Encontrar los tres SAPs con más consultas
top_saps = consultas_por_sap.head(3)

# Encontrar el SAP con menos consultas
sap_menos_consultas = consultas_por_sap.tail(1)

# Crear el texto con la información solicitada
texto_informacion = f"Los tres SAPs con más consultas:\n"
for i, row in top_saps.iterrows():
    texto_informacion += f"{row['nombre_correcto']} - {int(row['consulta_cantidad'])} pacientes\n"

texto_informacion += f"\nEl SAP con menos consultas:\n"
texto_informacion += f"{sap_menos_consultas.iloc[0]['nombre_correcto']} - {int(sap_menos_consultas.iloc[0]['consulta_cantidad'])} pacientes"

# Crear el gráfico Plotly
fig = px.bar(consultas_por_sap, x='nombre_correcto', y='consulta_cantidad',
             title='Consultas por SAP (Ordenado de Mayor a Menor)',
             labels={'nombre_correcto': 'SAP', 'consulta_cantidad': 'Cantidad de Consultas'})

fig.update_xaxes(title_text='SAP')
fig.update_yaxes(title_text='Cantidad de Consultas')

# Mostrar el texto en la consola
print(texto_informacion)

# Mostrar el gráfico interactivo
fig.show()

# Guardar la imagen en formato PNG para PowerPoint
fig.write_image("grafico_consultas.png")


Los tres SAPs con más consultas:
SAP XV - Dr. Semper - 8502 pacientes
SAP XXXIII - Dr. Sussini - 8026 pacientes
SAP XXV - Dr. Lifchitz - 7715 pacientes

El SAP con menos consultas:
SAP XIII - Bº Santa Margarita - 81 pacientes


In [429]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# ... (procesa los datos como se mostró anteriormente)

# Calcular la cantidad de consultas por SAP
consultas_por_sap = df.groupby('nombre_correcto')['consulta_cantidad'].sum().reset_index()

# Crear una columna que indique si el SAP pertenece a barrios populares o no
consultas_por_sap['es_barrio_popular'] = consultas_por_sap['nombre_correcto'].apply(lambda x: 'Barrio Popular' if x in df[df['barrio_popular'] == 'si']['nombre_correcto'].unique() else 'No Barrio Popular')

# Ordenar el DataFrame de mayor a menor cantidad de consultas
consultas_por_sap = consultas_por_sap.sort_values(by='consulta_cantidad', ascending=False)

# Encontrar los tres SAPs con más consultas
top_saps = consultas_por_sap.head(3)

# Encontrar el SAP con menos consultas
sap_menos_consultas = consultas_por_sap.tail(1)

# Crear el gráfico Plotly con colores diferentes para los SAPs de barrios populares
fig = px.bar(consultas_por_sap, x='nombre_correcto', y='consulta_cantidad',
             title='Consultas por SAP (Ordenado de Mayor a Menor)',
             labels={'nombre_correcto': 'SAP', 'consulta_cantidad': 'Cantidad de Consultas'},
             color='es_barrio_popular',  # Usar la columna 'es_barrio_popular' para asignar colores
            )  # Asignar colores a las categorías

fig.update_xaxes(title_text='SAP')
fig.update_yaxes(title_text='Cantidad de Consultas')

# Mostrar el gráfico interactivo
fig.show()

# Guardar la imagen en formato PNG para PowerPoint
fig.write_image("grafico_consultas_colores.png")

In [430]:
import pandas as pd
import plotly.express as px

# Supongamos que tienes tu DataFrame llamado 'df' con los datos
# ... (procesa los datos como se mostró anteriormente)

# Calcular la cantidad de consultas por SAP
consultas_por_sap = df.groupby(['nombre_correcto', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Crear una columna que indique si el SAP pertenece a un barrio popular o no
consultas_por_sap['es_barrio_popular'] = consultas_por_sap['barrio_popular'].apply(lambda x: 'Barrio Popular' if x == 'si' else 'No Barrio Popular')

# Ordenar el DataFrame de mayor a menor cantidad de consultas
consultas_por_sap = consultas_por_sap.sort_values(by='consulta_cantidad', ascending=False)

# Encontrar los tres SAPs con más consultas
top_saps = consultas_por_sap.head(3)

# Encontrar el SAP con menos consultas
sap_menos_consultas = consultas_por_sap.tail(1)

# Crear el gráfico Plotly con colores diferentes para los SAPs de barrios populares
fig = px.bar(consultas_por_sap, x='nombre_correcto', y='consulta_cantidad',
             title='Consultas por SAP (Ordenado de Mayor a Menor)',
             labels={'nombre_correcto': 'SAP', 'consulta_cantidad': 'Cantidad de Consultas'},
             color='es_barrio_popular',  # Usar la columna 'es_barrio_popular' para asignar colores
            )  # Asignar colores a las categorías

fig.update_xaxes(title_text='SAP')
fig.update_yaxes(title_text='Cantidad de Consultas')

# Mostrar el gráfico interactivo
fig.show()

# Guardar la imagen en formato PNG para PowerPoint
fig.write_image("grafico_consultas_colores.png")


In [454]:
df

,saps,fecha,servicio,turno,consulta_cantidad,rango_etario,sexo,detalle
0,DR. BENJAMIN SERRANO,2020-02-01,ODONTOLOGIA,TARDE,5.0,1 - 4 ANIOS,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...
1,DR. BENJAMIN SERRANO,2020-02-01,ODONTOLOGIA,TARDE,8.0,1 - 4 ANIOS,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...
2,DR. BENJAMIN SERRANO,2020-02-01,ODONTOLOGIA,TARDE,15.0,5 - 14 ANIOS,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...
3,DR. BENJAMIN SERRANO,2020-02-01,ODONTOLOGIA,TARDE,14.0,5 - 14 ANIOS,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...
4,DR. BENJAMIN SERRANO,2020-02-01,ODONTOLOGIA,TARDE,1.0,15 - 19 ANIOS,F,CONSULTA-ADULTO (Desde 15 anios)
...,...,...,...,...,...,...,...,...
22974,DR. MANUEL CASSUSO,2023-01-31,ODONTOLOGIA,MANANA,1.0,< 1 ANO,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...
22975,DR. MANUEL CASSUSO,2023-01-31,ODONTOLOGIA,MANANA,5.0,1 - 4 ANOS,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...
22976,DR. MANUEL CASSUSO,2023-01-31,ODONTOLOGIA,MANANA,3.0,1 - 4 ANOS,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...
22977,DR. MANUEL CASSUSO,2023-01-31,ODONTOLOGIA,MANANA,2.0,5 - 14 ANOS,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...


In [461]:
import pandas as pd
import plotly.express as px

# Cargar los datos en un DataFrame
data = pd.read_csv('dataframe_saps_limpio.csv')
# Eliminar filas con valor "#¡REF!" en la columna 'consulta_cantidad'
data = data[data['sexo'] != '#¡REF!']
data = data[data['rango_etario_normalizado'] != 'Desconocido']

# 1. Distribución por Sexo
sexo_distribution = data['sexo'].value_counts()
fig1 = px.bar(sexo_distribution, x=sexo_distribution.index, y=sexo_distribution.values, title='Distribución por Sexo')
fig1.show()

# 3. Consulta Promedio por Sexo
consulta_promedio_por_sexo = data.groupby('sexo')['consulta_cantidad'].mean().reset_index()
fig3 = px.bar(consulta_promedio_por_sexo, x='sexo', y='consulta_cantidad', title='Consulta Promedio por Sexo')
fig3.show()


# 5. Consulta Promedio por Tipo de Consulta (Bar Chart)
consulta_promedio_por_tipo = data.groupby('detalle')['consulta_cantidad'].mean().reset_index()
fig5 = px.bar(consulta_promedio_por_tipo, x='detalle', y='consulta_cantidad', title='Consulta Promedio por Tipo de Consulta')
fig5.update_xaxes(categoryorder='total descending')  # Ordenar las categorías por cantidad
fig5.show()

# 6. Consultas por Fecha y Turno
data['fecha'] = pd.to_datetime(data['fecha'])  # Convertir la columna 'fecha' a formato de fecha
consulta_por_fecha_turno = data.groupby(['fecha', 'turno'])['consulta_cantidad'].sum().unstack(fill_value=0)
fig6 = px.bar(consulta_por_fecha_turno, x=consulta_por_fecha_turno.index, title='Consultas por Fecha y Turno')
fig6.show()

# Puedes ajustar la apariencia de los gráficos según tus preferencias.

In [457]:
# 1. Distribución por Sexo
sexo_distribution = data['sexo'].value_counts()
fig1 = px.bar(sexo_distribution, x=sexo_distribution.index, y=sexo_distribution.values, title='Distribución por Sexo')
fig1.show()

In [462]:
data

,Unnamed: 0,fecha,nombre_correcto,servicio,turno,rango_etario_normalizado,consulta_cantidad,sexo,detalle,nombre_barrio_correcto,barrio_popular,sup_ha,lng,lat,st_asgeojson
0,0,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
1,1,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,1-4 ANOS,8.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
2,2,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,15.0,F,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
3,3,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,5-14 ANOS,14.0,M,CONSULTAS FICHADAS Y MOTIVACION (ninios hasta ...,PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
4,4,2020-02-01,SAP XVII - Bº Parque Cadenas Norte,ODONTOLOGIA,TARDE,15-19 ANOS,1.0,F,CONSULTA-ADULTO (Desde 15 anios),PARQUE CADENA NORTE,no,"32,76074115",-58.751160,-27.490823,"{""type"":""Polygon"",""coordinates"":[[[-58.7520693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22971,22974,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,<1 ANO,1.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22972,22975,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,5.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22973,22976,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,1-4 ANOS,3.0,M,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."
22974,22977,2023-01-31,SAP IX - Dr. M. Casusso,ODONTOLOGIA,MANANA,5-14 ANOS,2.0,F,CONSULTAS FICHADAS Y MOTIVACION (NINOS hasta 1...,SAN ANTONIO,no,"74,53253449",-58.813192,-27.507874,"{""type"":""Polygon"",""coordinates"":[[[-58.8196322..."


In [464]:
import pandas as pd
import plotly.express as px

# Cargar los datos en un DataFrame
data = pd.read_csv('dataframe_saps_limpio.csv')
# Eliminar filas con valor "#¡REF!" en la columna 'consulta_cantidad'
data = data[data['sexo'] != '#¡REF!']
data = data[data['rango_etario_normalizado'] != 'Desconocido']

# 1. Distribución por Sexo (Pie chart)
sexo_distribution = data['sexo'].value_counts()
fig1 = px.pie(sexo_distribution, names=sexo_distribution.index, title='Distribución por Sexo')
fig1.show()

# 2. Distribución por Rango Etario
etario_distribution = data['rango_etario_normalizado'].value_counts()
fig2 = px.bar(etario_distribution, x=etario_distribution.index, y=etario_distribution.values, title='Distribución por Rango Etario')
fig2.show()

# 3. Consulta Promedio por Sexo
consulta_promedio_por_sexo = data.groupby(['sexo', 'barrio_popular'])['consulta_cantidad'].mean().reset_index()
fig3 = px.bar(consulta_promedio_por_sexo, x='sexo', y='consulta_cantidad', color='barrio_popular', title='Consulta Promedio por Sexo (dividido por Barrio Popular)')
fig3.show()

# 4. Tipo de Consulta por Sexo y Barrio Popular (Heatmap)
tipo_consulta_sexo_barrio = data.groupby(['sexo', 'barrio_popular', 'detalle'])['detalle'].count().unstack(fill_value=0)
fig4 = px.imshow(tipo_consulta_sexo_barrio, x=tipo_consulta_sexo_barrio.columns, y=tipo_consulta_sexo_barrio.index, title='Tipo de Consulta por Sexo y Barrio Popular')
fig4.show()

# 5. Consulta Promedio por Tipo de Consulta (Bar Chart)
consulta_promedio_por_tipo = data.groupby(['detalle', 'barrio_popular'])['consulta_cantidad'].mean().reset_index()
fig5 = px.bar(consulta_promedio_por_tipo, x='detalle', y='consulta_cantidad', color='barrio_popular', title='Consulta Promedio por Tipo de Consulta (dividido por Barrio Popular)')
fig5.update_xaxes(categoryorder='total descending')  # Ordenar las categorías por cantidad
fig5.show()

# 6. Consultas por Fecha y Turno
data['fecha'] = pd.to_datetime(data['fecha'])  # Convertir la columna 'fecha' a formato de fecha
consulta_por_fecha_turno = data.groupby(['fecha', 'turno', 'barrio_popular'])['consulta_cantidad'].sum().unstack(fill_value=0)
fig6 = px.bar(consulta_por_fecha_turno, x=consulta_por_fecha_turno.index, color='barrio_popular', title='Consultas por Fecha y Turno (dividido por Barrio Popular)')
fig6.show()


TypeError: Argument 'x' is a pandas MultiIndex. pandas MultiIndex is not supported by plotly express at the moment.

In [467]:
# 1. Distribución por Sexo (Pie chart)
data['sexo'].value_counts()

F    12298
M    10638
Name: sexo, dtype: int64

In [469]:
import pandas as pd
import plotly.express as px

# Cargar los datos en un DataFrame
data = pd.read_csv('dataframe_saps_limpio.csv')
# Eliminar filas con valor "#¡REF!" en la columna 'consulta_cantidad'
data = data[data['sexo'] != '#¡REF!']
data = data[data['rango_etario_normalizado'] != 'Desconocido']

# 1. Distribución por Sexo (Pie chart)
sexo_distribution = data['sexo'].value_counts()
fig1 = px.pie(sexo_distribution, names=sexo_distribution.index, title='Distribución por Sexo', color_discrete_sequence=px.colors.qualitative.Set1)
fig1.show()

In [488]:
import plotly.express as px
import plotly.graph_objects as go

# Agrupar los datos por 'sexo' y sumar la 'consulta_cantidad'
genero = df.groupby('sexo')['consulta_cantidad'].sum().reset_index()
genero = genero[genero['sexo'] != '#¡REF!']

# Mapear las etiquetas 'M' y 'F' a 'Masculino' y 'Femenino'
genero['sexo'] = genero['sexo'].map({'M': 'Masculino', 'F': 'Femenino'})

# Convertir la columna 'consulta_cantidad' a enteros
genero['consulta_cantidad'] = genero['consulta_cantidad'].astype(int)

# Crear un gráfico de pie
fig = px.pie(
    genero,
    names='sexo',  # Usar 'sexo' como etiquetas en el gráfico de pie
    values='consulta_cantidad',  # Usar 'consulta_cantidad' como valores en el gráfico de pie
    title='Distribución por Género',  # Título del gráfico
    template='plotly',  # Usar el template estándar de Plotly
)

# Agregar etiquetas de cantidad y porcentaje personalizadas al gráfico
cantidad_y_porcentaje = genero['consulta_cantidad'].astype(str) + ' (' + (genero['consulta_cantidad'] / genero['consulta_cantidad'].sum() * 100).round(2).astype(str) + '%)'
fig.update_traces(
    text=cantidad_y_porcentaje,
    textposition='inside',
    hoverinfo='value+percent'
)

# Mostrar el gráfico
fig.show()

In [497]:
import plotly.express as px

# Agrupar los datos por 'sexo' y 'barrio_popular' y sumar la 'consulta_cantidad'
genero_barrio = data.groupby(['sexo', 'barrio_popular'])['consulta_cantidad'].sum().reset_index()

# Crear un gráfico de barras
fig = px.bar(
    genero_barrio,
    x='barrio_popular',  # Barrios en el eje X
    y='consulta_cantidad',  # Cantidad de consultas en el eje Y
    color='sexo',  # Colorear por sexo
    labels={'consulta_cantidad': 'Cantidad de Consultas'},
    title='Distribución de Consultas por Sexo y Barrio Popular',
)

# Agregar etiquetas de cantidad en las barras
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Mostrar el gráfico
fig.show()

In [505]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'data' que contiene tus datos

# Agrupar los datos por 'sexo' y 'servicio' y sumar la 'consulta_cantidad'
consulta_sexo_tipo = data.groupby(['sexo', 'detalle'])['consulta_cantidad'].sum().reset_index()

consulta_sexo_tipo = consulta_sexo_tipo[consulta_sexo_tipo['detalle'] != '#¡REF!']
consulta_sexo_tipo = consulta_sexo_tipo[consulta_sexo_tipo['detalle'] != 'CONSULTA-ADULTO (Desde 15 anios)']
consulta_sexo_tipo = consulta_sexo_tipo[consulta_sexo_tipo['detalle'] != 'CONSULTA-ADULTO (Desde 15 anos)']


# Crear un gráfico de barras apiladas
fig = px.bar(
    consulta_sexo_tipo,
    x='servicio',  # Tipo de consulta en el eje X
    y='consulta_cantidad',  # Cantidad de consultas en el eje Y
    color='sexo',  # Colorear por sexo
    labels={'consulta_cantidad': 'Cantidad de Consultas'},
    title='Distribución de Consultas por Sexo y Tipo de Consulta (Barras Apiladas)',
    barmode='group',  # Barras apiladas en grupos
)

# Mostrar el gráfico
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['sexo', 'detalle', 'consulta_cantidad'] but received: servicio

In [503]:
import plotly.express as px

# Supongamos que ya tienes un DataFrame llamado 'consulta_sexo_tipo' con los datos agrupados



ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['sexo', 'detalle', 'consulta_cantidad'] but received: servicio